In [2]:
import pandas as pd

In [3]:
with open('sample_files/ISL69269-0 0x60f.txt') as file:
    file1_contents = file.read()

In [4]:
with open('sample_files/ISL69269-0 0x60g.txt') as file:
    file2_contents = file.read()

In [5]:
lines1        = file1_contents.split('\n')
lines2        = file2_contents.split('\n')

commands1     = lines1[9:-1]
commands2     = lines2[9:-1]

In [148]:
class Config_cmdline:
    def __init__(self,line):
        self.line = line
        self.name = self.line[:20].rstrip()
        self.value = self.line[37:47].rstrip()
        self.addr  = self.line[61:].rstrip()
        
        self.add_leading_0byte_if_r()  
        
    def add_leading_0byte_if_r(self):
        if self.name[0] == 'r' and len(self.addr)==2:
            self.addr='00'+self.addr #.append('00') #is trailing since has been reversed by data_to_list()
        
def create_command_dict(commands):
    line   = [x for x,line in enumerate(commands)]
    name   = [Config_cmdline(line).name  for line in commands]
    value  = [Config_cmdline(line).value for line in commands]
    addr   = [Config_cmdline(line).addr  for line in commands]
    return {'line':line,'name':name, 'value': value, 'address': addr}
    
def create_diff_df_with_pagecalls(df1,df2):
    def add_missing_pagecalls(df_1,df_2,df_delta):
        df_delta_with_pages=df_delta.loc[:,['line','name','value','address']]
        for row in df_delta.index:
            line,name,value,addr = df_delta.loc[row,['line','name','value','address']]
            is_PMB_address=len(addr)==2
            if is_PMB_address:
                df_closest_preceeding_pageline = df_2[df_2['name']=='PAGE'][df2['line']<line].tail(1)
                df_delta_with_pages=df_delta_with_pages.append(df_closest_preceeding_pageline)
        df_delta_with_pages=df_delta_with_pages.sort_values(by=['line'])
        return df_delta_with_pages
    df_diff = df1.merge(df2,how='outer',indicator=True).loc[lambda x : x['_merge']=='right_only']
    df_diff = add_missing_pagecalls(df1,df2,df_diff)
    return df_diff

In [7]:
command1_dict = create_command_dict(commands1)
command2_dict = create_command_dict(commands2)

In [8]:
len(command2_dict['address'][136])

4

In [9]:
df1 = pd.DataFrame(command1_dict) #,columns=['value','address'],index=command1_dict['name'])
df2 = pd.DataFrame(command2_dict) #,columns=['value','address'],index=command1_dict['name'])

In [150]:
delta = create_diff_df_with_pagecalls(df1,df2)
delta

<ipython-input-148-68d0dfc49940>:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_closest_preceeding_pageline = df_2[df_2['name']=='PAGE'][df2['line']<line].tail(1)
<ipython-input-148-68d0dfc49940>:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_closest_preceeding_pageline = df_2[df_2['name']=='PAGE'][df2['line']<line].tail(1)


,line,name,value,address
1008,136,rGUI5,E4C06063,003F
288,288,PAGE,00,0
1009,289,oocLimit,03E8,46
480,480,PAGE,00,0
1010,481,slowOcLimit,04E2,EA


In [10]:
#[df1[df1['name']=='PAGE'].line  < 181]
#df1[df1['name']=='PAGE'][df1['line']<181].index[-1]   #.loc[180,['line','name','value','address']]
df1[df1['name']=='PAGE'][df1['line']<181].iloc[-1].value

<ipython-input-10-cbe12fbdaf26>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1[df1['name']=='PAGE'][df1['line']<181].iloc[-1].value


'00'

In [11]:
#df1[df1['name']=='PAGE'and df1['line']==180]
#page = df1[df1['line']==180]#.loc['value']
df1.loc[180,['name','value','address']]#[df1.line==180].value


name       PAGE
value        00
address       0
Name: 180, dtype: object

In [12]:
df_diff1=df1.merge(df2, how = 'outer',indicator=True).loc[lambda x : x['_merge']=='left_only']
df_diff1

,line,name,value,address,_merge
136,136,rGUI5,A1A08E17,003F,left_only
289,289,oocLimit,05DC,46,left_only
481,481,slowOcLimit,05DC,EA,left_only


In [87]:
df_diff2=df1.merge(df2,how='outer',indicator=True).loc[lambda x : x['_merge']=='right_only']
df_diff2.iloc[:,0:4]  #[0,1,2,3]]

,line,name,value,address
1008,136,rGUI5,E4C06063,003F
1009,289,oocLimit,03E8,46
1010,481,slowOcLimit,04E2,EA


In [86]:
df_diff2.loc[:,['line','name','value','address']]

,line,name,value,address
1008,136,rGUI5,E4C06063,003F
1009,289,oocLimit,03E8,46
1010,481,slowOcLimit,04E2,EA


In [14]:
a,b,c = df2.loc[288,['name','value','address']]
a

'PAGE'

In [91]:
newpage=df2[df2['name']=='PAGE'][df2['line']<289].tail(1)#.iloc[-1]
newpage

<ipython-input-91-fb6a99325168>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  newpage=df2[df2['name']=='PAGE'][df2['line']<289].tail(1)#.iloc[-1]


,line,name,value,address
288,288,PAGE,00,0


In [103]:
new_page=newpage.iloc[0,1]
new_page

'PAGE'

In [89]:
pd.concat([df_diff2.loc[:,['line','name','value','address']],df2[df2['name']=='PAGE'][df2['line']<289].tail(1)])

<ipython-input-89-bc2388bf6211>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pd.concat([df_diff2.loc[:,['line','name','value','address']],df2[df2['name']=='PAGE'][df2['line']<289].tail(1)])


,line,name,value,address
1008,136,rGUI5,E4C06063,003F
1009,289,oocLimit,03E8,46
1010,481,slowOcLimit,04E2,EA
288,288,PAGE,00,0


In [130]:
l,a=df_diff2.loc[:,['line','address']]
l,a

('line', 'address')

In [133]:
l=df_diff2.iloc[:,0]
l

1008    136
1009    289
1010    481
Name: line, dtype: int64

In [107]:
df_diff2.index[0]

1008

In [144]:
def add_missing_pagecalls(dataframe):
    df_diff_with_pages=df_diff2.loc[:,['line','name','value','address']]
    for row in df_diff2.index:
        line,name,value,addr = df_diff2.loc[row,['line','name','value','address']]
        is_PMB_address=len(addr)==2
        if is_PMB_address:
            df_closest_preceeding_pageline = df2[df2['name']=='PAGE'][df2['line']<line].tail(1)
            df_diff_with_pages=df_diff_with_pages.append(df_closest_preceeding_pageline)
    df_diff_with_pages=df_diff_with_pages.sort_values(by=['line'])
    return df_diff_with_pages

<ipython-input-144-8d7f824dfcde>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_closest_preceeding_pageline = df2[df2['name']=='PAGE'][df2['line']<line].tail(1)
<ipython-input-144-8d7f824dfcde>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_closest_preceeding_pageline = df2[df2['name']=='PAGE'][df2['line']<line].tail(1)


,line,name,value,address
1008,136,rGUI5,E4C06063,003F
288,288,PAGE,00,0
1009,289,oocLimit,03E8,46
480,480,PAGE,00,0
1010,481,slowOcLimit,04E2,EA


In [145]:
def add_missing_pagecalls(df_1,df_2,df_diff):
    df_diff_with_pages=df_diff.loc[:,['line','name','value','address']]
    for row in df_diff.index:
        line,name,value,addr = df_diff.loc[row,['line','name','value','address']]
        is_PMB_address=len(addr)==2
        if is_PMB_address:
            df_closest_preceeding_pageline = df_2[df_2['name']=='PAGE'][df2['line']<line].tail(1)
            df_diff_with_pages=df_diff_with_pages.append(df_closest_preceeding_pageline)
    df_diff_with_pages=df_diff_with_pages.sort_values(by=['line'])
    return df_diff_with_pages

In [147]:
df_d=add_missing_pagecalls(df1,df1,df_diff2)
df_d

<ipython-input-145-91d15e7f13ef>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_closest_preceeding_pageline = df_2[df_2['name']=='PAGE'][df2['line']<line].tail(1)
<ipython-input-145-91d15e7f13ef>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_closest_preceeding_pageline = df_2[df_2['name']=='PAGE'][df2['line']<line].tail(1)


,line,name,value,address
1008,136,rGUI5,E4C06063,003F
288,288,PAGE,00,0
1009,289,oocLimit,03E8,46
480,480,PAGE,00,0
1010,481,slowOcLimit,04E2,EA


In [141]:
df_diff_with_pages.sort_values(by=['line'])

,line,name,value,address
1008,136,rGUI5,E4C06063,003F
288,288,PAGE,00,0
1009,289,oocLimit,03E8,46
480,480,PAGE,00,0
1010,481,slowOcLimit,04E2,EA


In [140]:
l=df_diff_with_pages['line'].to_list()
l[0].type
#df_diff_with_pages.reindex(l)

AttributeError: 'int' object has no attribute 'type'

In [64]:
df2['name'].equals(df1['name'])

True

In [65]:
df2['address'].equals(df1['address'])

True

In [66]:
pd.concat([df1,df2]).drop_duplicates(keep=False)


,name,value,address
136,rGUI5,A1A08E17,3F
289,oocLimit,05DC,46
481,slowOcLimit,05DC,EA
136,rGUI5,E4C06063,3F
289,oocLimit,03E8,46
481,slowOcLimit,04E2,EA


In [67]:
set(df1.value).symmetric_difference(df2.value)

{'04E2', '05DC', 'A1A08E17', 'E4C06063'}

In [104]:
new_page

'PAGE'

In [108]:
test=newpage
test

,line,name,value,address
288,288,PAGE,00,0


In [115]:
test=test.append(test)

In [116]:
test

,line,name,value,address
288,288,PAGE,00,0
288,288,PAGE,00,0
288,288,PAGE,00,0
288,288,PAGE,00,0
288,288,PAGE,00,0
288,288,PAGE,00,0
288,288,PAGE,00,0
288,288,PAGE,00,0
